In [5]:
#importing requests to get API data
import requests
import json

#importing pandas
import pandas as pd

In [36]:
#script to compile data into dataframe per symbol
def get_keys(path):
    with open(path) as f:
        return json.load(f)

keys = get_keys("/Users/akharina/.secret/alpha_vantage_API.json")
api_key = keys['api_key']

def vantage_point(symbol):
    URL = 'https://www.alphavantage.co/query?'
    PARAMS = {"function":'TIME_SERIES_DAILY',
              "symbol":f'{symbol}',
              "outputsize":'full',
              "apikey":api_key}
    
    response = requests.get(URL,PARAMS)
    print(f"{symbol} response code: {response.status_code}")
    df = pd.DataFrame(response.json()['Time Series (Daily)']).T
    df.columns = [f'{symbol}_open',f'{symbol}_high',f'{symbol}_low',f'{symbol}_close',f'{symbol}_volume']
    df.to_csv(f"data/raw/{symbol}.csv")
    return df

df_slv=vantage_point('SLV')

SLV response code: 200


In [16]:
df_slv.head()

,SLV_open,SLV_high,SLV_low,SLV_close,SLV_volume
2019-10-04,16.3000,16.4600,16.2500,16.4100,17178596
2019-10-03,16.3800,16.6500,16.3700,16.4300,24412700
2019-10-02,16.3300,16.5400,16.2700,16.4300,21300800
2019-10-01,15.9600,16.2100,15.9100,16.1500,23278100
2019-09-30,16.0300,16.1000,15.8300,15.9200,29218300


In [24]:
#Compile data for all symbols
df_slv = vantage_point('SLV')
df_sil = vantage_point('SIL')
df_gld = vantage_point('GLD')
df_gdx = vantage_point('GDX')
df_dji = vantage_point('DJI')

SLV response code: 200
SIL response code: 200
GLD response code: 200
GDX response code: 200
DJI response code: 200


In [25]:
df_slv.shape,df_sil.shape,df_gld.shape,df_gdx.shape,df_dji.shape

((3383, 5), (2383, 5), (3745, 5), (3367, 5), (4971, 5))

In [31]:
df_dji.tail()

,DJI_open,DJI_high,DJI_low,DJI_close,DJI_volume
2000-01-07,11247.0596,11528.1396,11239.9199,11522.5596,184900000
2000-01-06,11113.3701,11313.4502,11098.4502,11253.2598,176550000
2000-01-05,10989.3701,11215.0996,10938.6699,11122.6504,203190000
2000-01-04,11349.7500,11350.0596,10986.4502,10997.9297,178420000
2000-01-03,11501.8496,11522.0098,11305.6904,11357.5098,169750000


In [32]:
#merge all datasets starting from largest

df_dji_gld = df_dji.join(df_gld,how='left')
df_bullion = df_dji_gld.join(df_slv,how='left')
df_all = df_bullion.join(df_gdx,how='left').join(df_sil,how='left')

In [33]:
df_all.isna().sum()

DJI_open         0
DJI_high         0
DJI_low          0
DJI_close        0
DJI_volume       0
GLD_open      1226
GLD_high      1226
GLD_low       1226
GLD_close     1226
GLD_volume    1226
SLV_open      1588
SLV_high      1588
SLV_low       1588
SLV_close     1588
SLV_volume    1588
GDX_open      1604
GDX_high      1604
GDX_low       1604
GDX_close     1604
GDX_volume    1604
SIL_open      2588
SIL_high      2588
SIL_low       2588
SIL_close     2588
SIL_volume    2588
dtype: int64